# 5 突破時空限制–整合搜尋功能

**準備工作**

請依照第 2 章說明上傳 .env 檔

In [ ]:
!pip install python-dotenv
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
!pip install openai
import openai

## 5-1 用搜尋網頁幫 AI 補充知識

### 使用 Google 搜尋

In [ ]:
!pip install googlesearch-python

In [ ]:
from googlesearch import search

In [ ]:
for item in search("NBA 2023 冠軍隊"):
    print(item)

使用進階搜尋選項

In [ ]:
for item in search(
    "NBA 2023 冠軍隊", advanced=True, num_results=3):
    print(item.title)
    print(item.description)
    print(item.url)
    print()

## 5-2 整合搜尋結果讓 AI 跟上時代

加入網頁搜尋的聊天程式

In [ ]:
def get_reply_s(messages):
    try:
        response = openai.ChatCompletion.create(
            model = "gpt-3.5-turbo",
            messages = messages,
            stream = True
        )
        for chunk in response:
            if 'content' in chunk['choices'][0]['delta']:
                yield chunk["choices"][0]["delta"]["content"]
    except openai.OpenAIError as err:
        reply = f"發生 {err.error.type} 錯誤\n{err.error.message}"

In [ ]:
hist = []       # 歷史對話紀錄
backtrace = 2   # 記錄幾組對話

def chat_w(sys_msg, user_msg):
    web_res = []
    if user_msg[:3].lower() == '/w ': # /w 代表要搜尋網路
        user_msg = user_msg[3:]       # 移除指令留下實際的訊息
        content = "以下為已發生的事實：\n"
        for res in search(user_msg, advanced=True,
                          num_results=5, lang='zh-TW'):
            content += f"標題：{res.title}\n" \
                       f"摘要：{res.description}\n\n"
        content += "請依照上述事實回答以下問題：\n"
        web_res = [{"role": "user", "content": content}]
    web_res.append({"role": "user", "content": user_msg})
    while len(hist) >= 2 * backtrace: # 超過記錄限制
        hist.pop(0)  # 移除最舊的紀錄
    reply_full = ""
    for reply in get_reply_s(         # 使用串流版的函式
        hist                          # 先提供歷史紀錄
        + web_res                     # 再提供搜尋結果及目前訊息
        + [{"role": "system", "content": sys_msg}]):
        reply_full += reply           # 記錄到目前為止收到的訊息
        yield reply                   # 傳回本次收到的片段訊息
    hist.append({"role": "user", "content": user_msg})
    while len(hist) >= 2 * backtrace: # 超過記錄限制
        hist.pop(0)                   # 移除最舊紀錄
    hist.append({"role":"assistant", "content":reply_full})

In [ ]:

sys_msg = input("你希望ㄟ唉扮演：")
if not sys_msg.strip(): sys_msg = '使用繁體中文的小助理'
print()
while True:
    msg = input("你說：")
    if not msg.strip(): break
    print(f"{sys_msg}：", end = "")
    for reply in chat_w(sys_msg, msg):
        print(reply, end = "")
    print('\n')
hist = []

## 5-3 使用 Google Search JSON AP

### 使用 HTTP API 取得搜尋結果

In [ ]:
import requests

In [ ]:
r = requests.get(
    'https://www.googleapis.com/customsearch/v1?' \
    'key={}&' \
    'cx={}&' \
    'num={}&' \
    'q={}'.format(
        '你的 Google API Key',
        '你的搜尋引擎 ID',
        2,
        '2023 NBA 冠軍'
    )
)

In [ ]:
r.json()

In [ ]:
for item in r.json()['items']:
    print(item['title'])
    print(item['snippet'])
    print(item['link'])
    print()

### 使用客製模組

In [ ]:
!git clone https://github.com/codemee/customsearchapi.git \
customsearchapi

fatal: destination path 'customsearchapi' already exists and is not an empty directory.


In [ ]:
# 請在你的 .env 檔中加入以下兩行：
# GOOGLE_API_KEY=你的 Google API 金鑰
# GOOGLE_CSE_ID=你的搜尋引擎 ID
load_dotenv()

True

In [ ]:
from customsearchapi import search
# 預設會在匯入時從環境變數 GOOGLE_API_KEY 與 GOOGLE_ID
# 讀取你的 API Key 與搜尋引擎 ID,
# 如果沒有設定, 也可以直接透過模組內的變數設定：
# customsearchapi.GOOGLE_API_KEY = '你的 Google API 金鑰'
# customsearchapi.GOOGLE_CSE_ID = '你的搜尋引擎 ID'

In [ ]:
for item in search("2023 NBA 冠軍", advanced=True, num_results=3):
    print(item.url)
    print(item.title)
    print(item.description)
    print()

https://zh.wikipedia.org/zh-hant/2023%E5%B9%B4NBA%E7%B8%BD%E6%B1%BA%E8%B3%BD
2023年NBA總決賽- 維基百科，自由的百科全書
2023年NBA總決賽（英語：2023 NBA Finals）是2022–23 NBA賽季的冠軍系列賽，將由2023年6月1日至6月12日進行，由西區第一種子丹佛金塊對戰東區第八種子邁阿密熱火，比賽 ...

https://www.womenshealthmag.com/tw/fitness/work-outs/g43820597/2023-nba/
2023NBA總冠軍賽程：G5「金塊94： 熱火89」金塊抱得總冠軍獎杯 ...
比賽已經進入尾聲！把握機會啊～. By Avis Wu. 2023/06/12. 2023nba季後賽賽程：總冠軍賽「金塊vs 熱火」每日賽程. Getty Images. 2022-23 NBA 總冠軍賽(National ...

https://zh.wikipedia.org/zh-cn/2023%E5%B9%B4NBA%E7%B8%BD%E6%B1%BA%E8%B3%BD
2023年NBA总决赛- 维基百科，自由的百科全书
2023年NBA总决赛（英语：2023 NBA Finals）是2022–23 NBA赛季的冠军系列赛，将由2023年6月1日至6月12日进行，由西部第一种子丹佛掘金对战东部第八种子迈阿密热火，比赛 ...

